# DDL

Etablissez d'abord une connexion à la base 'DS2' sur le serveur dans votre VM

In [ ]:
-- connection: host='localhost' dbname='ds2' user='ds2' 

----------------------------------------------------------------------------

## Exercices sur les 3 tables exemples

* Exécutez la commande suivante afin d'avoir les 3 tables modèles

In [ ]:
DROP TYPE IF EXISTS lp_level CASCADE;

CREATE TYPE lp_level AS ENUM ('gold', 'silver', 'bronze'); 

DROP TABLE IF EXISTS loyalty_program;

CREATE TABLE loyalty_program ( 
custId INTEGER NOT NULL, 
lpLevel lp_level 
); 

DROP TABLE IF EXISTS lp_cust;

CREATE TABLE lp_cust ( 
custId SERIAL NOT NULL, 
firstName CHARACTER VARYING(50), 
lastName CHARACTER VARYING(50), 
userName CHARACTER VARYING(50), 
CONSTRAINT lp_cust_pkey PRIMARY KEY (custId) 
); 

DROP TABLE IF EXISTS lp_actor;

CREATE TABLE lp_actor ( 
actorId SERIAL NOT NULL, 
firstName CHARACTER VARYING(50), 
lastName CHARACTER VARYING(50), 
CONSTRAINT lp_act_pkey PRIMARY KEY (actorId) 
);

* Exécutez la commande suivante afin de remplir les tables

In [ ]:
INSERT INTO lp_cust VALUES 
(1, 'Fred' ,'Flinstone' ,'freddo' ), 
(2, 'John' ,'Smith' ,'jsmith' ), 
(3, 'Homer' ,'Simpson' ,'homey' ), 
(4, 'Homer' ,'Brown' ,'notsofamous' ), 
(5, 'Ozzy' ,'Ozzbourn' ,'sabbath' ), 
(6, 'Homer' ,'Gain' ,'noplacelike' ), 
(7, 'Jack' ,'Brown' ,'bigjack' ), 
(8, 'Morten' ,'Harket' ,'aha' ), 
(9, 'John' ,'Smith' ,'jsmith1' ); 

INSERT INTO loyalty_program VALUES 
(1, 'gold' ), 
(2, 'silver' ), 
(3, 'silver' ), 
(4, 'bronze' ), 
(5, 'bronze' ), 
(10, 'silver' ), 
(10, 'bronze' ), 
(11, 'gold' ); 

INSERT INTO lp_actor VALUES 
(1, 'Fred' ,'Flinstone' ), 
(2, 'Homer' ,'Simpson' ), 
(3, 'Greta' ,'Garbo' ), 
(4, 'Jennifer' , 'Lawrence' ), 
(5, 'Will' ,'Smith' );

* Réalisez entre les 2 premières tables tous les types de jointure vues en cours.

In [ ]:
SELECT *
FROM lp_cust A
INNER JOIN loyalty_program B ON A.custId = B.custId;

In [ ]:
SELECT *
FROM lp_cust A
LEFT JOIN loyalty_program B ON A.custId = B.custId;

In [ ]:
SELECT *
FROM lp_cust A
RIGHT JOIN loyalty_program B ON A.custId = B.custId;

In [ ]:
SELECT *
FROM lp_cust A
INNER JOIN lp_cust B ON A.custId = B.custId;

In [ ]:
SELECT *
FROM lp_cust A
FULL OUTER JOIN loyalty_program B ON A.custId = B.custId;

In [ ]:
SELECT *
FROM lp_cust
CROSS JOIN loyalty_program;

In [ ]:
SELECT *                              -- INNER JOIN est appliqué par defaut
FROM lp_cust
NATURAL JOIN loyalty_program;

* Réalisez entre les 2 tables compatibles les opérations suivantes dans les 2 sens : __UNION, UNION ALL, INTERSECT, EXECEPT__

In [ ]:
SELECT firstName, lastName             -- ENLEVE LES DOUBLONS
FROM lp_cust                           -- L'ORDRE DES TABLES DANS LA REQUETE 
UNION                                  -- N'EST PAS PERTINENT
SELECT firstName, lastName
FROM lp_actor;

In [ ]:
SELECT firstName, lastName             -- LAISSE LES DOUBLONS
FROM lp_cust                           -- L'ORDRE DES TABLES DANS LA REQUETE 
UNION ALL                              -- N'EST PAS PERTINENT
SELECT firstName, lastName
FROM lp_actor;

In [ ]:
SELECT firstName, lastName             -- LIGNES COMMUNES
FROM lp_cust                           -- L'ORDRE DES TABLES DANS LA REQUETE 
INTERSECT                              -- N'EST PAS PERTINENT
SELECT firstName, lastName
FROM lp_actor;

In [ ]:
SELECT firstName, lastName             -- LIGNES UNIQUEMENT PRESENTES DANS LA PREMIERE TABLE
FROM lp_cust                          -- L'ORDRE DES TABLES DANS LA REQUETE 
EXCEPT                                 -- N'EST PAS PERTINENT
SELECT firstName, lastName
FROM lp_actor;

In [ ]:
SELECT firstName, lastName             -- LIGNES UNIQUEMENT PRESENTES DANS LA PREMIERE TABLE
FROM lp_actor                          -- L'ORDRE DES TABLES DANS LA REQUETE 
EXCEPT                                 -- N'EST PAS PERTINENT
SELECT firstName, lastName
FROM lp_cust;

----------------------------------------------------------------------------

## Exercices sur la base de test

* Affichez le nombre de clients qui n'ont pas passé de commande et le nombre de clients total

In [ ]:
SELECT count(*)
FROM customers
LEFT JOIN orders ON orders.customerid = customers.customerid
WHERE orderid IS null
UNION 
SELECT count(*)
FROM customers;

* Sélectionnez les produits vendus en plus de 30 exemplaires (_trié par nombre d'exemplaires_)

In [ ]:
SELECT products.title, sum(quantity)
FROM products
LEFT JOIN orderlines ON products.prod_id = orderlines.prod_id
GROUP BY products.title
HAVING sum(quantity) > 30
ORDER BY sum(quantity) DESC;

* Listez pour chaque produit la quantité en stock et la quantité commandée
</br> (_Nous allons trier le résultat suivant la quantité en stock_)
</br> (_De plus, le résultat retourne 50 000 lignes, On ne va afficher que les 20 premierères_)

In [ ]:
SELECT title, sum(quan_in_stock), sum(quantity)
FROM products
LEFT JOIN inventory ON products.prod_id = inventory.prod_id
LEFT JOIN orderlines ON products.prod_id = orderlines.prod_id
GROUP BY title
ORDER BY sum(quan_in_stock) DESC
LIMIT 20;

* Comparez les quantité vendues et celles en stock
</br>_Nous reprenons la requête précédente et nous appliquons un pourcentage entre les 2 sommes qui indiquera la qualité de la gestion du stock pour se produit._
</br>_Certaines quantités en stock sont nulles, nous devons les traiter indépendamment afin d'éviter une division par 0 (qui conduit à une erreur et un arrêt du traitement)._
</br>_Dans notre exemple, ces cas là afficheront un ratio de -1_
</br> (_De plus, le résultat retourne 50 000 lignes, On ne va afficher que les 20 premierères_)

In [ ]:
select title, sum(quan_in_stock), sum(quantity), (cast(sum(quantity) as float)/cast(sum(quan_in_stock) as float))*100 AS RATIO
from products
left join inventory on products.prod_id = inventory.prod_id
left join orderlines on products.prod_id = orderlines.prod_id
group by title
HAVING max(quan_in_stock) <> 0
UNION
select title, max(quan_in_stock), sum(quantity), -1 AS RATIO
from products
left join inventory on products.prod_id = inventory.prod_id
left join orderlines on products.prod_id = orderlines.prod_id
group by title
HAVING max(quan_in_stock) = 0
LIMIT 20